# Multivariate analyses & simple phylogenetic linear regression

## Cleaning and loading data

### Loading packages

In [1]:
library("phytools")
library("convevol")
library("TreeTools")
library("nlme")
library("car")
library("factoextra")
library("ggrepel")
library("FactoMineR")
library("MASS")
library("stringr")
library("dplyr")

Le chargement a nécessité le package : ape

Le chargement a nécessité le package : maps

Le chargement a nécessité le package : MASS

Le chargement a nécessité le package : cluster


Attachement du package : ‘cluster’


L'objet suivant est masqué depuis ‘package:maps’:

    votes.repub


Le chargement a nécessité le package : geiger


Attachement du package : ‘TreeTools’


L'objet suivant est masqué depuis ‘package:phytools’:

    as.multiPhylo


Le chargement a nécessité le package : carData

Le chargement a nécessité le package : ggplot2

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa


Attachement du package : ‘dplyr’


L'objet suivant est masqué depuis ‘package:car’:

    recode


L'objet suivant est masqué depuis ‘package:nlme’:

    collapse


L'objet suivant est masqué depuis ‘package:MASS’:

    select


L'objet suivant est masqué depuis ‘package:ape’:

    where


Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag


### Loading data

In [2]:
phy<-read.tree(file = "data/Tree_65_sp_murids.tree")

In [3]:
multi_phy<-read.tree("data/TreeDist_murids.tree")

In [4]:
table_data<-read.table("data/Complete_Table.tsv")

## Simple phylogenetic regression

### 

In [5]:
tab_aggregated <- table_data[!duplicated(table_data$Nomenclature),][,c(1:5)]
for (i in c("TailLength", "Proxy_size_raw", "Proxy_size_GM", "RImean", "RIproxmean", "RItranmean", "RIdistmean", "RIlqmean", "TPEImean", "TPEIproxmean", "TPEItranmean", "TPEIdistmean", "TPEIlqmean")){
    temp_tab <- eval(parse(text = paste("aggregate(", i, " ~ Nomenclature, data = table_data, FUN = mean)", sep = "")))
    tab_aggregated <- cbind(tab_aggregated, temp_tab[,2])
}
colnames(tab_aggregated)[6:18] <- c("TailLength", "Proxy_size_raw", "Proxy_size_GM", "RImean", "RIproxmean", "RItranmean", "RIdistmean", "RIlqmean", "TPEImean", "TPEIproxmean", "TPEItranmean", "TPEIdistmean", "TPEIlqmean")

In [6]:
bm <- corBrownian(1, phy, form = ~Genus_species)

In [7]:
phylo_reg <- function(table, var, table_res){
    print(var)
    eval(parse(text = paste("Pgls_results <- gls(log(", var, ")~log(Proxy_size_GM), data=table, correlation=bm)", sep ="")))
    print(eval(parse(text = paste(
    "plot(log(", var, ")~log(Proxy_size_GM),data=table,cex=1.5,pch=19,bg='grey', col=c('#D2B48C','#40826D')[as.factor(table$Prehensility_lit)])
    text(log(", var, ")~log(Proxy_size_GM),data=table, labels=table$Nomenclature, cex = 0.6, pos = 1)
    abline(Pgls_results,lwd=2,col='darkgrey',lty='dashed')", sep = "")
    )))
    print(summary(Pgls_results))
    temp_table_res <- summary(Pgls_results)
    if (temp_table_res$tTable[2,4] < 0.05){
        residuals_phy_reg<- as.matrix(residuals(Pgls_results))
        colnames(residuals_phy_reg) <- paste(var, "resphy", sep = "")
        residuals_phy_reg
        return(cbind(table_res, residuals_phy_reg))
    }
    else{
        temp_col_non_res <- as.matrix(eval(parse(text = paste("table$", var, sep = ""))))
        colnames(temp_col_non_res) <- paste(var, sep = "")
        return(cbind(table_res, log(temp_col_non_res)))
    }
}

In [ ]:
table_res <- table_data[,c(1:6)]
for (i in c("TailLength", "RImean", "RIproxmean", "RItranmean", "RIdistmean", "RIlqmean", "TPEImean", "TPEIproxmean", "TPEItranmean", "TPEIdistmean", "TPEIlqmean")){
    table_res <- phylo_reg(table = table_data, var = i, table_res = table_res)
}

### Testing for normality

In [ ]:
for(i in 7:17){
    res_test<-shapiro.test(table_res[,i])
    if(res_test$p < 0.05){
        print(colnames(table_res)[i])
    }
}

## Principal Component Analysis

### All Vertebrae

In [ ]:
DF_PCA_all <- table_res[,c(6, 7, 9, 10, 11, 14, 15, 16)]
rownames(DF_PCA_all) <- table_res$Genus_species
PCA_all <- PCA(DF_PCA_all)
fviz_pca_var(PCA_all, axes = c(1, 2), label="all")
fviz_pca_ind(PCA_all, axes = c(1, 2), label="all", habillage=as.factor(table_res$Prehensility), addEllipses = TRUE)

In [ ]:
dataset_all <-data.frame(size=table_data$Proxy_size_GM, PC1=PCA_all$ind$coord[,1],PC2=PCA_all$ind$coord[,2],PC3=PCA_all$ind$coord[,3])
size<-dataset_all$size
cores<-c("#D2B48C","#40826D")
PC_all12 <- ggplot(dataset_all) + geom_point(aes(x=PC1,y=PC2,color=table_data$Prehensility,size=size,shape=table_res$Climbing)) + scale_color_manual(values=cores,aesthetics = c("colour","fill")) + theme(text = element_text(size=15), legend.title = element_text(colour="black", size=15, face = "bold"),legend.background = element_rect(fill="lightgray",size=0.5, linetype = "solid", colour = "darkgray")) + scale_x_continuous(name = paste("PC1 (", round(PCA(DF_PCA_all)$eig[1,2], 1), "%)", sep = "")) + scale_y_continuous(name = paste("PC2 (", round(PCA(DF_PCA_all)$eig[2,2], 1), "%)", sep = "")) + geom_text_repel(aes(x=PC1,y=PC2,label=table_res$Nomenclature),size=2)
PC_all13 <- ggplot(dataset_all) + geom_point(aes(x=PC1,y=PC3,color=table_data$Prehensility,size=size,shape=table_res$Climbing))  + scale_color_manual(values=cores,aesthetics = c("colour","fill")) + theme(text = element_text(size=15), legend.title = element_text(colour="black", size=15, face = "bold"),legend.background = element_rect(fill="lightgray",size=0.5, linetype = "solid", colour = "darkgray")) + scale_x_continuous(name = paste("PC1 (", round(PCA(DF_PCA_all)$eig[1,2], 1), "%)", sep = "")) + scale_y_continuous(name = paste("PC3 (", round(PCA(DF_PCA_all)$eig[3,2], 1), "%)", sep = "")) + geom_text_repel(aes(x=PC1,y=PC3,label=table_res$Nomenclature),size=2)

In [ ]:
PC_all12
PC_all13

### Distal Vertebrae

In [ ]:
DF_PCA_distal <- table_res[,c(6, 7, 11, 16)]
rownames(DF_PCA_distal) <- table_res$Genus_species
PCA_distal <- PCA(DF_PCA_distal)
fviz_pca_var(PCA_distal, axes = c(1, 2), label="distal")
fviz_pca_ind(PCA_distal, axes = c(1, 2), label="distal", habillage=as.factor(table_res$Prehensility), addEllipses = TRUE)

In [ ]:
dataset_distal <-data.frame(size=table_data$Proxy_size_GM, PC1=PCA_distal$ind$coord[,1],PC2=PCA_distal$ind$coord[,2],PC3=PCA_distal$ind$coord[,3])
size<-dataset_distal$size
cores<-c("#D2B48C","#40826D")
PC_distal12 <- ggplot(dataset_distal) + geom_point(aes(x=PC1,y=PC2,color=table_data$Prehensility,size=size,shape=table_res$Climbing)) + scale_color_manual(values=cores,aesthetics = c("colour","fill")) + theme(text = element_text(size=15), legend.title = element_text(colour="black", size=15, face = "bold"),legend.background = element_rect(fill="lightgray",size=0.5, linetype = "solid", colour = "darkgray")) + scale_x_continuous(name = paste("PC1 (", round(PCA(DF_PCA_distal)$eig[1,2], 1), "%)", sep = "")) + scale_y_continuous(name = paste("PC2 (", round(PCA(DF_PCA_distal)$eig[2,2], 1), "%)", sep = "")) + geom_text_repel(aes(x=PC1,y=PC2,label=table_res$Nomenclature),size=2)
PC_distal13 <- ggplot(dataset_distal) + geom_point(aes(x=PC1,y=PC3,color=table_data$Prehensility,size=size,shape=table_res$Climbing))  + scale_color_manual(values=cores,aesthetics = c("colour","fill")) + theme(text = element_text(size=15), legend.title = element_text(colour="black", size=15, face = "bold"),legend.background = element_rect(fill="lightgray",size=0.5, linetype = "solid", colour = "darkgray")) + scale_x_continuous(name = paste("PC1 (", round(PCA(DF_PCA_distal)$eig[1,2], 1), "%)", sep = "")) + scale_y_continuous(name = paste("PC3 (", round(PCA(DF_PCA_distal)$eig[3,2], 1), "%)", sep = "")) + geom_text_repel(aes(x=PC1,y=PC3,label=table_res$Nomenclature),size=2)

In [ ]:
PC_distal12
PC_distal13

### Last Quarter

In [ ]:
DF_PCA_last_Q <- table_res[,c(6, 7, 12, 17)]
rownames(DF_PCA_last_Q) <- table_res$Genus_species
PCA_last_Q <- PCA(DF_PCA_last_Q)
fviz_pca_var(PCA_last_Q, axes = c(1, 2), label="last_Q")
fviz_pca_ind(PCA_last_Q, axes = c(1, 2), label="last_Q", habillage=as.factor(table_res$Prehensility), addEllipses = TRUE)

In [ ]:
dataset_last_Q <-data.frame(size=table_data$Proxy_size_GM, PC1=PCA_last_Q$ind$coord[,1],PC2=PCA_last_Q$ind$coord[,2],PC3=PCA_last_Q$ind$coord[,3])
size<-dataset_last_Q$size
cores<-c("#D2B48C","#40826D")
PC_last_Q12 <- ggplot(dataset_last_Q) + geom_point(aes(x=PC1,y=PC2,color=table_data$Prehensility,size=size,shape=table_res$Climbing)) + scale_color_manual(values=cores,aesthetics = c("colour","fill")) + theme(text = element_text(size=15), legend.title = element_text(colour="black", size=15, face = "bold"),legend.background = element_rect(fill="lightgray",size=0.5, linetype = "solid", colour = "darkgray")) + scale_x_continuous(name = paste("PC1 (", round(PCA(DF_PCA_last_Q)$eig[1,2], 1), "%)", sep = "")) + scale_y_continuous(name = paste("PC2 (", round(PCA(DF_PCA_last_Q)$eig[2,2], 1), "%)", sep = "")) + geom_text_repel(aes(x=PC1,y=PC2,label=table_res$Nomenclature),size=2)
PC_last_Q13 <- ggplot(dataset_last_Q) + geom_point(aes(x=PC1,y=PC3,color=table_data$Prehensility,size=size,shape=table_res$Climbing))  + scale_color_manual(values=cores,aesthetics = c("colour","fill")) + theme(text = element_text(size=15), legend.title = element_text(colour="black", size=15, face = "bold"),legend.background = element_rect(fill="lightgray",size=0.5, linetype = "solid", colour = "darkgray")) + scale_x_continuous(name = paste("PC1 (", round(PCA(DF_PCA_last_Q)$eig[1,2], 1), "%)", sep = "")) + scale_y_continuous(name = paste("PC3 (", round(PCA(DF_PCA_last_Q)$eig[3,2], 1), "%)", sep = "")) + geom_text_repel(aes(x=PC1,y=PC3,label=table_res$Nomenclature),size=2)

In [ ]:
PC_last_Q12
PC_last_Q13

## Linear Discriminant analysis

### Training group creation

In [ ]:
training_group <- c("Thamnomys_poensis","Hyomys_goliath","Hyomys_goliath_2","Mallomys_hercules","Mallomys_istapantap","Mallomys_rothschildi","Protochromys_fellowsi","Uromys_imperator","Uromys_rex")

### All Vertebrae

### Data preparation

In [ ]:
test.data_all <- DF_PCA_all[training_group,]
codeprehLDAphylo <- as.factor(table_data$Prehensility)
names(codeprehLDAphylo) = table_data$Genus_species
codeprehLDAphylo_training <- codeprehLDAphylo[names(codeprehLDAphylo) %in% setdiff(names(codeprehLDAphylo), training_group)]
training.data_all <- DF_PCA_all[rownames(DF_PCA_all) %in% names(codeprehLDAphylo[names(codeprehLDAphylo) %in% setdiff(names(codeprehLDAphylo), training_group)]),]

### LDA

In [ ]:
model_all <- lda(training.data_all, codeprehLDAphylo_training)
predictions_all <- model_all %>% predict(test.data_all)

### Distal Vertebrae

### Data preparation

In [ ]:
test.data_distal <- DF_PCA_distal[training_group,]
codeprehLDAphylo <- as.factor(table_data$Prehensility)
names(codeprehLDAphylo) = table_data$Genus_species
codeprehLDAphylo_training <- codeprehLDAphylo[names(codeprehLDAphylo) %in% setdiff(names(codeprehLDAphylo), training_group)]
training.data_distal <- DF_PCA_distal[rownames(DF_PCA_distal) %in% names(codeprehLDAphylo[names(codeprehLDAphylo) %in% setdiff(names(codeprehLDAphylo), training_group)]),]

### LDA

In [ ]:
model_distal <- lda(training.data_distal, codeprehLDAphylo_training)
predictions_distal <- model_distal %>% predict(test.data_distal)

### Last Quarter

### Data preparation

In [ ]:
test.data_last_Q <- DF_PCA_last_Q[training_group,]
codeprehLDAphylo <- as.factor(table_data$Prehensility)
names(codeprehLDAphylo) = table_data$Genus_species
codeprehLDAphylo_training <- codeprehLDAphylo[names(codeprehLDAphylo) %in% setdiff(names(codeprehLDAphylo), training_group)]
training.data_last_Q <- DF_PCA_last_Q[rownames(DF_PCA_last_Q) %in% names(codeprehLDAphylo[names(codeprehLDAphylo) %in% setdiff(names(codeprehLDAphylo), training_group)]),]

### LDA

In [ ]:
model_last_Q <- lda(training.data_last_Q, codeprehLDAphylo_training)
predictions_last_Q <- model_last_Q %>% predict(test.data_last_Q)

### Save estimated Prehensility type

#### Assuming a threshold at 70 %

In [ ]:
predictions_last_Q_posterior <- as.data.frame(predictions_last_Q$posterior)
Prehensility_LDA <- table_data$Prehensility
for(j in 1:nrow(predictions_last_Q_posterior)){
    if(predictions_last_Q_posterior[j,1] > 0.7){
        Prehensility_LDA[which(table_data$Genus_species == rownames(predictions_last_Q_posterior)[j])] <- "N"
    }
    if(predictions_last_Q_posterior[j,2] > 0.7){
        Prehensility_LDA[which(table_data$Genus_species == rownames(predictions_last_Q_posterior)[j])] <- "P"
    }
    if(predictions_last_Q_posterior[j,1] < 0.7 & predictions_last_Q_posterior[j,2] < 0.7){
        Prehensility_LDA[which(table_data$Genus_species == rownames(predictions_last_Q_posterior)[j])] <- NA
    }
}

In [ ]:
df_prehensility_ASE<-as.data.frame(cbind(table_data$Genus_species, Prehensility_LDA))

In [ ]:
write.table(df_prehensility_ASE, "data/Code_P_Muridae.tsv", sep="\t", col.names = TRUE, row.names = FALSE)

In [ ]:
write.table(as.data.frame(cbind(table_data, Prehensility_LDA)), "data/table_complete_LDA.tsv", sep="\t", col.names = TRUE, row.names = FALSE)

## Phylomorphospace & Phenogram

### Pre-painting the clades

In [ ]:
Prehensility_LDA <- as.character(df_prehensility_ASE$Prehensility_LDA)
Prehensility_LDA[which(is.na(Prehensility_LDA))] <- "UKN"
Prehensility_LDA <- as.factor(Prehensility_LDA)
names(Prehensility_LDA) <- table_data$Genus_species
colspheno<-setNames(c("#D2B48C","#40826D","#000000"), sort(unique(Prehensility_LDA)))
painted<-paintSubTree(phy,128,"PHL","0")
painted<-paintSubTree(painted,111,"MIC+RAT")
painted<-paintSubTree(painted,120,"VAN+MIL+PIT+ARV")
painted<-paintSubTree(painted,110,"CHI")
painted<-paintSubTree(painted,71,"HYD") 
paintedSC <- painted
coltree <- c("0"="black", "PHL"="#999999","MIC+RAT"="#D55E00", "VAN+MIL+PIT+ARV"="#0072B2", "CHI"="#56B4E9" ,"HYD"="#CC79A7")
cols<-c(rep("black",length(phy$tip.label)), rep("black", phy$Nnode))
names(cols)<-1:length(cols)
cols[df_prehensility_ASE$Prehensility_LDA=="N"]<-"#D2B48C"
cols[df_prehensility_ASE$Prehensility_LDA=="P"]<-"#40826D"

#### Phylomorphospace

#### All vertebrae

In [ ]:
tabphymor_all <- PCA_all$ind$coord[,1:3]
DFphymor_all <- data.frame(tabphymor_all,df_prehensility_ASE$Prehensility_LDA)
rownames(DFphymor_all) <- table_data$Genus_species

##### Plot

In [ ]:
phylmorph_all_12 <- phylomorphospace(paintedSC,DFphymor_all[,1:2],xlab="PC1",ylab="PC2", colors= coltree, control = list(col.node= cols), fsize=0.8, node.size=0)
points(DFphymor_all[,1:2][paintedSC$tip.label,],col="black", bg=cols ,pch=21,cex=1.6, lwd=0.9)

### Distal vertebrae

In [ ]:
tabphymor_distal <- PCA_distal$ind$coord[,1:3]
DFphymor_distal <- data.frame(tabphymor_distal,df_prehensility_ASE$Prehensility_LDA)
rownames(DFphymor_distal) <- table_data$Genus_species

##### Plot

In [ ]:
phylmorph_distal_12 <- phylomorphospace(paintedSC,DFphymor_distal[,1:2],xlab="PC1",ylab="PC2", colors= coltree, control = list(col.node= cols), fsize=0.8, node.size=0)
points(DFphymor_distal[,1:2][paintedSC$tip.label,],col="black", bg=cols ,pch=21,cex=1.6, lwd=0.9)

### Last Quarter vertebrae

In [ ]:
tabphymor_last_Q <- PCA_last_Q$ind$coord[,1:3]
DFphymor_last_Q <- data.frame(tabphymor_last_Q,df_prehensility_ASE$Prehensility_LDA)
rownames(DFphymor_last_Q) <- table_data$Genus_species

##### Plot

In [ ]:
phylmorph_last_Q_12 <- phylomorphospace(paintedSC,DFphymor_last_Q[,1:2],xlab="PC1",ylab="PC2", colors= coltree, control = list(col.node= cols), fsize=0.8, node.size=0)
points(DFphymor_last_Q[,1:2][paintedSC$tip.label,],col="black", bg=cols ,pch=21,cex=1.6, lwd=0.9)

##### Plot All (PC1)

In [ ]:
phenogram(painted,PCA_all$ind$coord[,1], colors = coltree,spread.labels=TRUE,spread.cost=c(1,0))
tiplabels(pie=to.matrix(Prehensility_LDA[painted$tip.label],seq=sort(unique(Prehensility_LDA))),
          piecol=colspheno,cex=0.2)

##### Plot All (PC2)

In [ ]:
phenogram(painted,PCA_all$ind$coord[,2], colors = coltree,spread.labels=TRUE,spread.cost=c(1,0))
tiplabels(pie=to.matrix(Prehensility_LDA[painted$tip.label],seq=sort(unique(Prehensility_LDA))),
          piecol=colspheno,cex=0.2)

##### Plot All (PC3)

In [ ]:
phenogram(painted,PCA_all$ind$coord[,3], colors = coltree,spread.labels=TRUE,spread.cost=c(1,0))
tiplabels(pie=to.matrix(Prehensility_LDA[painted$tip.label],seq=sort(unique(Prehensility_LDA))),
          piecol=colspheno,cex=0.2)

#### Plot Distal (PC1)

In [ ]:
phenogram(painted,PCA_distal$ind$coord[,1], colors = coltree,spread.labels=TRUE,spread.cost=c(1,0))
tiplabels(pie=to.matrix(Prehensility_LDA[painted$tip.label],seq=sort(unique(Prehensility_LDA))),
          piecol=colspheno,cex=0.2)

#### Plot Last Quarter (PC1)

In [ ]:
phenogram(painted,PCA_last_Q$ind$coord[,1], colors = coltree,spread.labels=TRUE,spread.cost=c(1,0))
tiplabels(pie=to.matrix(Prehensility_LDA[painted$tip.label],seq=sort(unique(Prehensility_LDA))),
          piecol=colspheno,cex=0.2)

## Convergence Index

## Data Preparation

In [ ]:
ScoresPC1 <- cbind(PCA_all$ind$coord[,1],PCA_distal$ind$coord[,1],PCA_last_Q$ind$coord[,1])
colnames(ScoresPC1) <- cbind("PC1_all","PC1_distal","PC1_last_Q")
scoresPC1_match <- ScoresPC1[match(phy$tip.label,rownames(ScoresPC1)),]
list_preh <- as.character(table_data$Genus_species[Prehensility_LDA == "P"])

data_convSig_all <- as.matrix(scoresPC1_match[,1])
data_convSig_distal <- as.matrix(scoresPC1_match[,2])
data_convSig_last_Q <- as.matrix(scoresPC1_match[,3])

### Convergence index model

#### All Vertebrae

##### PCI 

In [ ]:
calcConv(phy, scoresPC1_match[,1], list_preh, VERBOSE = FALSE)
resConv_all_PCI <- convSig(phy, data_convSig_all, list_preh, nsim = 1000)
saveRDS(resConv_all_PCI, "data/resConv_all_PCI.rds")

##### All axes

In [ ]:
mat_PCA_all <- data.matrix(DF_PCA_all)
calcConv(phy, mat_PCA_all, list_preh, VERBOSE = FALSE)
resConv_all <- convSig(phy, mat_PCA_all, list_preh, nsim = 1000)
saveRDS(resConv_all, "data/resConv_all.rds")

#### Distal Vertebrae

##### PCI 

In [ ]:
calcConv(phy, scoresPC1_match[,2], list_preh, VERBOSE = FALSE)
resConv_distalPCI <- convSig(phy, data_convSig_distal, list_preh, nsim = 1000)
saveRDS(resConv_distalPCI, "data/resConv_distalPCI.rds")

##### All axes

In [ ]:
mat_PCA_distal <- data.matrix(DF_PCA_distal)
calcConv(phy, mat_PCA_distal, list_preh, VERBOSE = FALSE)
resConv_distal <- convSig(phy, mat_PCA_distal, list_preh, nsim = 1000)
saveRDS(resConv_distal, "data/resConv_distal.rds")

#### Last Quarter Vertebrae

##### PCI 

In [ ]:
calcConv(phy, scoresPC1_match[,3], list_preh, VERBOSE = FALSE)
resConv_last_QPCI <- convSig(phy, data_convSig_last_Q, list_preh, nsim = 1000)
saveRDS(resConv_last_QPCI, "data/resConv_last_QPCI.rds")

##### All axes

In [ ]:
mat_PCA_last_Q <- data.matrix(DF_PCA_last_Q) 
calcConv(phy, mat_PCA_last_Q, list_preh, VERBOSE = FALSE)
resConv_last_Q <- convSig(phy, mat_PCA_last_Q, list_preh, nsim = 1000)
saveRDS(resConv_last_Q, "data/resConv_last_Q.rds")

### Optional (Convergence index on posterior tree distribution)

In [ ]:
for (i in 501:600){
    
phy<-multi_phy[[i]]

phy$tip.label<-paste(str_split_fixed(phy$tip.label, "_", 3)[,1], str_split_fixed(phy$tip.label, "_", 3)[,2], sep ="_")
phy<-AddTip(phy, where = which(phy$tip.label== "Hyomys_goliath"), label = "Hyomys_goliath_2", edgeLength = 0.1, lengthBelow = 0.1)
phy$tip.label[phy$tip.label=='Taeromys_punicans']<-'Taeromys_arcuatus'
phy$tip.label[phy$tip.label=='Lenothrix_canus']<-'Lenothrix_cana'
phy$tip.label[phy$tip.label=='Hydromys_obi']<-'Hydromys_chrisogaster'
phy$tip.label[phy$tip.label=='Leptomys_elegans']<-'Leptomys_sp'
phy$tip.label[phy$tip.label=='Microhydromys_richardsoni']<-'Microhydromys_sp'
phy$tip.label[phy$tip.label=='Mesembriomys_gouldii']<-'Mesembryomys_gouldii'
phy$tip.label[phy$tip.label=='Paramelomys_levipes']<-'Paramelomys_sp'
phy$tip.label[phy$tip.label=='Haeromys_minahassaeZ23000']<-'Haeromys_minahassae'
phy$tip.label[phy$tip.label=='Thallomys_sp']<-'Thallomys_spce'
phy$tip.label[phy$tip.label=='Pithecheir_chim']<-'Pithecheir_melanurus'


calcConv(phy, scoresPC1_match[,1], list_preh, VERBOSE = FALSE)
resConv_all_PCI <- convSig(phy, data_convSig_all, list_preh, nsim = 1000)
saveRDS(resConv_all_PCI, paste("conv_Index/resConv_allPCI_replicate_", i, ".rds", sep = ""))

##### All axes

mat_PCA_all <- data.matrix(DF_PCA_all)
calcConv(phy, mat_PCA_all, list_preh, VERBOSE = FALSE)
resConv_all <- convSig(phy, mat_PCA_all, list_preh, nsim = 1000)
saveRDS(resConv_all, paste("conv_Index/resConv_all_replicate_", i, ".rds", sep = ""))

#### Distal Vertebrae

##### PCI 

calcConv(phy, scoresPC1_match[,2], list_preh, VERBOSE = FALSE)
resConv_distalPCI <- convSig(phy, data_convSig_distal, list_preh, nsim = 1000)
saveRDS(resConv_distalPCI, paste("conv_Index/resConv_distalPCI_replicate_", i, ".rds", sep = ""))

##### All axes

mat_PCA_distal <- data.matrix(DF_PCA_distal)
calcConv(phy, mat_PCA_distal, list_preh, VERBOSE = FALSE)
resConv_distal <- convSig(phy, mat_PCA_distal, list_preh, nsim = 1000)
saveRDS(resConv_distal, paste("conv_Index/resConv_all_replicate_", i, ".rds", sep = ""))

#### Last Quarter Vertebrae

##### PCI 

calcConv(phy, scoresPC1_match[,3], list_preh, VERBOSE = FALSE)
resConv_last_QPCI <- convSig(phy, data_convSig_last_Q, list_preh, nsim = 1000)
saveRDS(resConv_last_QPCI, paste("conv_Index/resConv_last_Q_PCI_replicate_", i, ".rds", sep = ""))

##### All axes

mat_PCA_last_Q <- data.matrix(DF_PCA_last_Q) 
calcConv(phy, mat_PCA_last_Q, list_preh, VERBOSE = FALSE)
resConv_last_Q <- convSig(phy, mat_PCA_last_Q, list_preh, nsim = 1000)
saveRDS(resConv_last_Q, paste("conv_Index/resConv_last_Q_replicate_", i, ".rds", sep = ""))
}